In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "C:/Users/USER02/Documents/Voice2Text.json"

In [2]:
from __future__ import division

import re
import sys

from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import pyaudio
import time
import math
import audioop
from six.moves import queue

from google.rpc import code_pb2

In [3]:
flag_recogRepeat = True  # 音声認識を繰り返し行う場合　Trueにする
EXIT_WORD = u"(音声認識を終了します|ちちんぷいぷい|さようなら)"  # 音声認識を終了させる合言葉
LANG_CODE = 'ja-JP'  # a BCP-47 language tag

RATE = 16000  # サンプリングレート
CHANNELS = 1  # 録音チャンネル数

RECORD_SEC = 5  # 録音時間(sec)
DEV_INDEX = 0  # デバイスを指定

FRAME_SEC = 0.1  # 1フレームの時間（秒）　（0.1sec = 100ms）
CHUNK = int(RATE * FRAME_SEC)  # 1フレーム内のサンプルデータ数

SLEEP_SEC = FRAME_SEC / 4  # メインループ内でのスリープタイム（秒）
BUF_SIZE = CHUNK * 2  # 音声のバッファ・サイズ（byte）

DECIBEL_THRESHOLD = 50  # 録音開始のための閾値（dB)
START_FRAME_LEN = 4  # 録音開始のために，何フレーム連続で閾値を超えたらいいか
START_BUF_LEN = 5  # 録音データに加える，閾値を超える前のフレーム数

In [4]:
# バッファ用変数 #####################
frames = []
frames_startbuf = []

flag_RecordStart = False  # 音量が規定フレーム分，閾値を超え続けたらTRUE
flag_RecogEnd = False  # 音声認識が終わったらTrueにする

recog_result = ""  # 音声認識結果

In [5]:
# コールバック関数 ###################
def callback(in_data, frame_count, time_info, status):
    frames.append(in_data)
    return (None, pyaudio.paContinue)

In [6]:
def listen_print_loop(recognize_stream):
    global flag_RecogEnd
    global recog_result
    for resp in recognize_stream:
        
        if not resp.results:
            continue

        # 音声認識結果＆途中結果の表示 (受け取るデータの詳細は以下を参照のこと)
        # https://cloud.google.com/speech/reference/rpc/google.cloud.speech.v1beta1#google.cloud.speech.v1beta1.SpeechRecognitionAlternative
        for result in resp.results:
            if result.is_final:
                print('is_final:'+ str(result.is_final))

            for alt in result.alternatives:
                print ('conf:' + str(alt.confidence) + " stab:" + str(result.stability))
                print ('trans:' + alt.transcript)
                recog_result = alt.transcript

            # 音声認識終了（is_final: True）
            if result.is_final:
                flag_RecogEnd = True
                return


In [7]:
def make_channel(host, port):
    ssl_channel = implementations.ssl_channel_credentials(None, None, None)
    creds = get_credentials().create_scoped([SPEECH_SCOPE])
    auth_header = (
        'Authorization',
        'Bearer ' + creds.get_access_token().access_token)
    auth_plugin = implementations.metadata_call_credentials(
        lambda _, cb: cb([auth_header], None),
        name='google_creds')

    composite_channel = implementations.composite_channel_credentials(
        ssl_channel, auth_plugin)

    return implementations.secure_channel(host, port, composite_channel)

In [8]:
def request_stream(channels=CHANNELS, rate=RATE, chunk=CHUNK):
    global flag_RecogEnd
    global LANG_CODE
    recognition_config = cloud_speech.RecognitionConfig(
        encoding='LINEAR16',  # raw 16-bit signed LE samples
        sample_rate=rate,  # the rate in hertz
        language_code=LANG_CODE,  # a BCP-47 language tag
    )
    streaming_config = cloud_speech.StreamingRecognitionConfig(
        config=recognition_config,
        interim_results=True, single_utterance=True
    )

    yield cloud_speech.StreamingRecognizeRequest(
        streaming_config=streaming_config)

    while True:
        time.sleep(SLEEP_SEC)

        if flag_RecogEnd:
            return

        # バッファにデータが溜まったら，データ送信
        if len(frames) > 0:
            data_1frame = frames.pop(0)
            data_l2s = b''.join(map(str, data_1frame))
            wf.writeframes(data_l2s)  # waveファイルに書き込み
            yield cloud_speech.StreamingRecognizeRequest(audio_content=data_l2s)  # google ASR


In [9]:
class MicrophoneStream(object):
    """
    Opens a recording stream as a generator yielding the audio chunks.
    音声のチャンクをyieldする録音のストリームをオープンする（？？？）
    """
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        # オーディオデータの「スレッドセーフバッファ」？？？を作成する。
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            
            # このAPIは、現在のところモノラル音声にしか対応してない。
            # https://goo.gl/z757pE
            channels=1, rate=self._rate,
            input=True, frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            
            # オーディオストリームを、バッファを埋めるために非同期で動かそう。
            # これは、呼び出しているスレッドがネットワークにリクエストをしている間などであっても
            # 入力デバイスのバッファがオーバーフロウを起こさないようにするため、必要である。
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        # ジェネレータに終了の信号を伝える。
        # じゃないと、ストリーミング認識のメソッドが、プロセスの終了を阻害してしまう。
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            # （日訳）get()を用いて、少なくともデータが1チャンク以上あるかを確認してください。
            # データのチャンク数がNoneを示すとき、音声ストリームは終了しています。
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            # （日訳）？？？
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break
            # b''は、バイト型で文字列を送信するという記号
            yield b''.join(data)

In [10]:
def request_stream(channels=CHANNELS, rate=RATE, chunk=CHUNK):
    global flag_RecogEnd
    global LANG_CODE
    recognition_config = cloud_speech.RecognitionConfig(
        encoding='LINEAR16',  # raw 16-bit signed LE samples
        sample_rate=rate,  # the rate in hertz
        language_code=LANG_CODE,  # a BCP-47 language tag
    )
    streaming_config = cloud_speech.StreamingRecognitionConfig(
        config=recognition_config,
        interim_results=True, single_utterance=True
    )

    yield cloud_speech.StreamingRecognizeRequest(
        streaming_config=streaming_config)

    while True:
        time.sleep(SLEEP_SEC)

        if flag_RecogEnd:
            return

        # バッファにデータが溜まったら，データ送信
        if len(frames) > 0:
            data_1frame = frames.pop(0)
            data_l2s = b''.join(map(str, data_1frame))
            wf.writeframes(data_l2s)  # waveファイルに書き込み
            yield cloud_speech.StreamingRecognizeRequest(audio_content=data_l2s)  # google ASR


In [11]:
def main():
    global frames
    global frames_startbuf
    print('Start Rec!')

    # pyaudioオブジェクトを作成 --------------------
    p = pyaudio.PyAudio()
    
    language_code = 'ja-JP'  # a BCP-47 language tag
    
    stream = p.open(format=p.get_format_from_width(2),
                    channels=CHANNELS,
                    rate=RATE,
                    input_device_index=DEV_INDEX,
                    input=True,
                    output=False,
                    frames_per_buffer=CHUNK,
                    stream_callback=callback)

    client = speech.SpeechClient()
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code)
    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        interim_results=True)
    
    while True:
    
        # フラグ初期化 ##################################
        flag_RecordStart = False  # 音量が規定フレーム分，閾値を超え続けたらTRUE
        flag_RecogEnd = False  # 音声認識が終わったらTrueにする
        # 録音開始までの処理 ##############################
        while not flag_RecordStart:
            time.sleep(SLEEP_SEC)

            # 促音用バッファが長過ぎたら捨てる（STARTフレームより更に前のデータを保存しているバッファ）
            if len(frames_startbuf) > START_BUF_LEN:
                del frames_startbuf[0:len(frames_startbuf) - START_BUF_LEN]

            # バッファにデータが溜まったら，録音開始するべきか判定 ---------
            if len(frames) > START_FRAME_LEN:
                # 1フレーム内の音量計算--------------------------------
                for i in range(START_FRAME_LEN):
                    # from IPython.core.debugger import Pdb; Pdb().set_trace()
                    data = frames[i]
                    rms = audioop.rms(data, 2)
                    decibel = 20 * math.log10(rms) if rms > 0 else 0
                    sys.stdout.write("\rrms %3d decibel %f" %(rms,decibel))
                    sys.stdout.flush()

                    # 音量が閾値より小さかったら，データを捨てループを抜ける ----
                    if decibel < DECIBEL_THRESHOLD:
                        frames_startbuf.append(frames[0:i + 1])
                        del frames[0:i + 1]
                        break

                    # 全フレームの音量が閾値を超えていたら，録音開始！！ ----
                    # 更に，framesの先頭に，先頭バッファをプラス
                    # これをしないと「かっぱ」の「かっ」など，促音の前の音が消えてしまう
                    if i == START_FRAME_LEN - 1:
                        flag_RecordStart = True
                        frames = frames_startbuf + frames

        # googleサーバに接続 ############################
        print ("\nconnecting ....")
        with MicrophoneStream(RATE, CHUNK) as stream:
            audio_generator = stream.generator()
            requests = (types.StreamingRecognizeRequest(audio_content=content)
                        for content in audio_generator)

            responses = client.streaming_recognize(streaming_config, requests)
        # 録音開始後の処理 ###############################
            listen_print_loop(
            service.StreamingRecognize(
                request_stream(), DEADLINE_SECS))

        # 音声認識 繰り返しの終了判定 #####################
        if re.match(EXIT_WORD, recog_result):
            print('Exiting..')
            break

        # 音声認識繰り返ししない設定 ######################
        if not flag_recogRepeat:
            break

    # ストリームを止めて，クローズ
    print ('Closing audio stream....')
    stream.stop_stream()
    stream.close()

    p.terminate()  # pyaudioオブジェクトを終了

    print ('End Rec!')


In [12]:
main()

Start Rec!
rms 342 decibel 50.680522
connecting ....


NameError: name 'service' is not defined